# English Premier League and the Ratings Percentage Index

This notebook uses python tools to automatically generate the English Premier League table ordered by the Ratings Percetage Index (RPI), using football match results data at [www.football-data.co.uk](http://www.football-data.co.uk/englandm.php). The RPI is a technique proposed by [The Tomkins Times](https://tomkinstimes.com/) subscriber Tim O'Brien to take account of the quality of opposition and is described [here](https://tomkinstimes.com/2016/11/comment-of-the-month-october-2016/). The solution is built into a simple web app.

The project uses [jupyter notebook](http://jupyter.org/index.html), [python](https://www.python.org/), [pandas](http://pandas.pydata.org/), [beautiful soup](https://www.crummy.com/software/BeautifulSoup/), [requests](http://docs.python-requests.org/en/master/), [spyre](https://github.com/adamhajari/spyre) and [heroku](https://www.heroku.com/).

#### Notebook Change Log

In [1]:
%%html
<! left align the change log table in next cell >
<style>
table {float:left}
</style>

| Date          | Change Description |
| :------------ | :----------------- |
| 17th November 2016 | Initial baseline |
| 30th October 2017 | Fixed error in calculation for P (number of games played) in *gen_prem_table_RPI()*|

## Set-up

Import the modules needed for the analysis.

In [27]:
import pandas as pd
import sys 
import requests
import datetime as dt
import pickle
import os
import bs4
from bs4 import BeautifulSoup, SoupStrainer
from itertools import cycle
from collections import defaultdict
from IPython.display import Image
from IPython.core.display import HTML 
from __future__ import division

# enable inline plotting
%matplotlib inline

Print version numbers of key modules.

In [28]:
print 'python version: {}'.format(sys.version)
print 'pandas version: {}'.format(pd.__version__)
print 'requests version: {}'.format(requests.__version__)
print 'bs4 version: {}'.format(bs4.__version__)
#print 'matplotlib version: {}'.format(mpl.__version__)
#print 'numpy version: {}'.format(np.__version__)

python version: 2.7.11 |Anaconda custom (64-bit)| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
pandas version: 0.18.0
requests version: 2.13.0
bs4 version: 4.4.1


## Generate the Premier League table with RPI

Start by defining some utility functions

In [29]:
def get_pl_master_data():
    """Return url of latest premier league results file and the date the file was last updated.
    
    Data source is www.football-data.co.uk.
    Format of returned date is "%Y-%m-%d" (the pandas default).
    """
    # scrape the data from football-data website
    URL_FD_ROOT = 'http://www.football-data.co.uk/'
    ENGLAND_LOCATION = 'englandm.php'
    PL_TEXT = 'Premier League'
    with requests.Session() as session:
        response = session.get(URL_FD_ROOT + ENGLAND_LOCATION)
        soup = BeautifulSoup(response.content, 'lxml')

        # scrape last updated date
        last_updated_tag = soup.find_all('i')[0]
        last_updated_date = last_updated_tag.text.split('Last updated: \t')[1]
        # set date format to be same as pandas default
        last_updated_date = dt.datetime.strptime(last_updated_date, '%d/%m/%y').strftime('%Y-%m-%d')
                                                                                         
        # scrape url of premier league results file
        latest_pl_results_file_tag = soup.findAll('a', href=True, text=PL_TEXT)[0]['href']
        url_latest_pl_results_file = URL_FD_ROOT + latest_pl_results_file_tag                                                                         
                                                                                     
    return(url_latest_pl_results_file, last_updated_date)

In [30]:
# check current latest
url_latest_pl_results_file, last_updated_date = get_pl_master_data()
print 'PL results URL: {}, last updated: {}'.format(url_latest_pl_results_file,  last_updated_date)

PL results URL: http://www.football-data.co.uk/mmz4281/1718/E0.csv, last updated: 2017-10-29


In [31]:
def get_pl_results_dataframe(update_cache=False):
    """Return latest premier league results as a dataframe and the date of the results data.
    
    Data source is www.football-data.co.uk.
    Cache data locally to avoid unnecessary calls to football-data website.
    Download results from master data source if local data is out of date.
    """
    
    LOCAL_RESULTS_DATA_FILE = 'data/E0.csv'
    PICKLE_FILE = 'save.p' # holds date of results data file

    # get master data source data
    url_latest_pl_results_file, master_results_data_date = get_pl_master_data()
    
    if update_cache:
        if os.path.exists(PICKLE_FILE):
            os.remove(PICKLE_FILE)

    # get local data source date
    if os.path.exists(PICKLE_FILE):
        local_results_data_date = pickle.load(open(PICKLE_FILE, 'rb'))
    else:
        local_results_data_date = None

    if local_results_data_date < master_results_data_date:
        print 'local results data out of date, updating from master results data file'
        parse_dates_col = ['Date']
        df_results = pd.read_csv(url_latest_pl_results_file, parse_dates=parse_dates_col, dayfirst=True)
        df_results.to_csv(LOCAL_RESULTS_DATA_FILE, index=False)
        local_results_data_date = master_results_data_date
        pickle.dump(local_results_data_date, open(PICKLE_FILE, 'wb'))
    else:
        print 'local results data still latest'
        parse_dates_col = ['Date']
        df_results = pd.read_csv(LOCAL_RESULTS_DATA_FILE, parse_dates=parse_dates_col, dayfirst=True)
        
    return df_results, local_results_data_date

In [32]:
# check results
df_results, results_date = get_pl_results_dataframe()
print 'results data date: {}'.format(results_date)
print df_results.dtypes.head()
df_results.head()

local results data still latest
results data date: 2017-10-29
Div                 object
Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG               float64
dtype: object


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4.0,3.0,H,2.0,2.0,D,...,2.3,21,-1.0,1.9,1.9,2.1,2.0,1.5,4.7,7.2
1,E0,2017-08-12,Brighton,Man City,0.0,2.0,A,0.0,0.0,D,...,2.3,20,1.5,1.9,1.9,2.0,2.0,11.8,6.2,1.3
2,E0,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,...,2.2,20,-1.8,2.0,2.0,1.9,1.9,1.3,5.4,12.2
3,E0,2017-08-12,Crystal Palace,Huddersfield,0.0,3.0,A,0.0,2.0,A,...,1.7,18,-0.8,2.1,2.0,1.9,1.8,1.8,3.6,5.5
4,E0,2017-08-12,Everton,Stoke,1.0,0.0,H,1.0,0.0,H,...,1.8,19,-0.8,1.9,1.9,2.0,2.0,1.8,3.5,5.4


In [33]:
# check forced update to cache
df_results, results_date = get_pl_results_dataframe(update_cache=True)
print 'results data date: {}'.format(results_date)
print df_results.dtypes.head()
df_results.head()

local results data out of date, updating from master results data file
results data date: 2017-10-29
Div                 object
Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG               float64
dtype: object


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4.0,3.0,H,2.0,2.0,D,...,2.3,21,-1.0,1.9,1.9,2.1,2.0,1.5,4.7,7.2
1,E0,2017-08-12,Brighton,Man City,0.0,2.0,A,0.0,0.0,D,...,2.3,20,1.5,1.9,1.9,2.0,2.0,11.8,6.2,1.3
2,E0,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,...,2.2,20,-1.8,2.0,2.0,1.9,1.9,1.3,5.4,12.2
3,E0,2017-08-12,Crystal Palace,Huddersfield,0.0,3.0,A,0.0,2.0,A,...,1.7,18,-0.8,2.1,2.0,1.9,1.8,1.8,3.6,5.5
4,E0,2017-08-12,Everton,Stoke,1.0,0.0,H,1.0,0.0,H,...,1.8,19,-0.8,1.9,1.9,2.0,2.0,1.8,3.5,5.4


In [34]:
# check forced reload
local_data_source_date = '2016-11-05'
PICKLE_FILE = 'save.p'
pickle.dump(local_data_source_date, open(PICKLE_FILE, 'wb'))
print pickle.load(open(PICKLE_FILE, 'rb'))
df_results, results_date = get_pl_results_dataframe()
print 'results data date: {}'.format(results_date)
df_results.head()

2016-11-05
local results data out of date, updating from master results data file
results data date: 2017-10-29


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4.0,3.0,H,2.0,2.0,D,...,2.3,21,-1.0,1.9,1.9,2.1,2.0,1.5,4.7,7.2
1,E0,2017-08-12,Brighton,Man City,0.0,2.0,A,0.0,0.0,D,...,2.3,20,1.5,1.9,1.9,2.0,2.0,11.8,6.2,1.3
2,E0,2017-08-12,Chelsea,Burnley,2.0,3.0,A,0.0,3.0,A,...,2.2,20,-1.8,2.0,2.0,1.9,1.9,1.3,5.4,12.2
3,E0,2017-08-12,Crystal Palace,Huddersfield,0.0,3.0,A,0.0,2.0,A,...,1.7,18,-0.8,2.1,2.0,1.9,1.8,1.8,3.6,5.5
4,E0,2017-08-12,Everton,Stoke,1.0,0.0,H,1.0,0.0,H,...,1.8,19,-0.8,1.9,1.9,2.0,2.0,1.8,3.5,5.4


In [35]:
df_results.tail()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
95,E0,2017-10-28,Watford,Stoke,0.0,1.0,A,0.0,1.0,A,...,1.9,19,-0.2,1.8,1.7,2.2,2.2,2.0,3.6,4.1
96,E0,2017-10-28,West Brom,Man City,2.0,3.0,A,1.0,2.0,A,...,2.5,19,1.8,2.0,1.9,2.0,1.9,11.0,6.4,1.3
97,E0,2017-10-29,Brighton,Southampton,1.0,1.0,D,0.0,1.0,A,...,1.5,18,0.2,1.9,1.8,2.1,2.1,3.4,3.3,2.4
98,E0,2017-10-29,Leicester,Everton,2.0,0.0,H,2.0,0.0,H,...,1.9,19,-0.2,1.9,1.8,2.1,2.0,2.1,3.5,4.0
99,E0,2017-10-30,Burnley,Newcastle,NaN,NaN,NaN,NaN,NaN,NaN,...,1.6,18,0.0,2.1,2.0,1.9,1.8,NaN,NaN,NaN


In [36]:
def validate_date(date_text):
    """Raise error if date format is not YYYY-MM-DD."""
    try:
        dt.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect date format, should be YYYY-MM-DD")

In [37]:
validate_date('2016-11-14')
#validate_date('14-11-2016') # produces error

In [38]:
def simple_date(date_text):
    """Return given date in format '%y-%m-%d' to '%d %b %y'."""
    validate_date(date_text)
    return (dt.datetime.strptime(date_text, '%Y-%m-%d').strftime('%d %b %y'))

In [39]:
print simple_date('2016-6-2')
print simple_date('2017-1-31')

02 Jun 16
31 Jan 17


Now produce function to generate the Prem League Table with RPI

Function Change Log
- October 2017, Fixed error in calculation for P (number of games played) if the game
is listed but not yet played, so game data is not available

In [40]:
def gen_prem_table_RPI(before_date=None, update_cache=False):
    """Return prem table with RPI at given before_date and return data source date."""
    
    results = []
    opponents_d = {}
    df_results, results_date = get_pl_results_dataframe(update_cache)
    
    
    # filter results in dataframe at given before_date
    if before_date:
        validate_date(before_date)
        df_results = df_results[df_results.Date <= before_date]
    
    for team in df_results['HomeTeam'].unique():
        home_results = df_results[df_results['HomeTeam'] == team]
        home_played = len(home_results.index)
        home_win = home_results.FTR[home_results.FTR == 'H'].count()
        home_draw = home_results.FTR[home_results.FTR == 'D'].count()
        home_lose = home_results.FTR[home_results.FTR == 'A'].count()
        home_goals_for = home_results.FTHG.sum()
        home_goals_against = home_results.FTAG.sum()
        home_opponents = list(df_results[df_results.HomeTeam == team].AwayTeam.values)

        away_results = df_results[df_results['AwayTeam'] == team]
        away_played = len(away_results.index)
        away_win = away_results.FTR[away_results.FTR == 'A'].count()
        away_draw = away_results.FTR[away_results.FTR == 'D'].count()
        away_lose = away_results.FTR[away_results.FTR == 'H'].count()
        away_goals_for = away_results.FTAG.sum()
        away_goals_against = away_results.FTHG.sum()
        away_opponents = list(df_results[df_results.AwayTeam == team].HomeTeam.values)

        # add team opponents to dictionary
        team_opponents = home_opponents + away_opponents
        opponents_d[team] = team_opponents
        
        # create team results dictionary and add to results list
        result_d = {} 
        result_d['Team'] = team
        result_d['W'] = home_win + away_win
        result_d['D'] = home_draw + away_draw
        result_d['L'] = home_lose + away_lose
        result_d['GF'] = home_goals_for + away_goals_for
        result_d['GA'] = home_goals_against + away_goals_against
        result_d['GD'] = result_d['GF'] - result_d['GA']
        result_d['PTS'] = result_d['W']*3 + result_d['D']
        result_d['P'] = result_d['W'] + result_d['D'] + result_d['L']
        results.append(result_d) # append team result dictionary to list of results

    # create PL table dataframe from team results and sort by points (and then goal difference and goals for)
    # show date of data in Position column
    PLtable = pd.DataFrame(results, columns=['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS'])
    PLtable.sort_values(['PTS', 'GD', 'GF'], ascending=False, inplace=True)
    col_date = before_date if before_date else results_date
    pos_title = 'Position at {}'.format(simple_date(col_date))
    PLtable[pos_title] = range(1, len(PLtable)+1) # add new column for position, with highest points first
    PLtable.set_index([pos_title], inplace=True, drop=True) 
    #PLtable.reset_index(inplace=True)
    
    # Add RPI to the table
    PLtable['PTS%'] = 100*(PLtable.PTS/(PLtable.P*3))
    PLtable['OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['PTS%'].mean(), axis=1)
    PLtable['OPP_OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['OPP_PTS%'].mean(), axis=1)
    PLtable['RPI'] = (PLtable['PTS%']*.25 + PLtable['OPP_PTS%']*.50 + PLtable['OPP_OPP_PTS%']*.25)
    PLtable['RPI_Position'] = PLtable['RPI'].rank(ascending=False).astype(int)
    
    # return PL table with RPI, sorted by RPI and PTS percentage
    return(PLtable.sort_values(['RPI', 'PTS%'], ascending=False), results_date)

In [41]:
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI()
print 'Premier league table ordered by RPI, date of results data: {}'.format(results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI, date of results data: 2017-10-29


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 29 Oct 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35.0,6.0,29.0,28,93.3,38.8,49.9,55.2,1
6,Liverpool,10,4,4,2,17.0,16.0,1.0,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18.0,10.0,8.0,19,63.3,46.1,47.3,50.7,3
3,Tottenham,10,6,2,2,19.0,7.0,12.0,20,66.7,44.0,46.8,50.4,4
2,Man United,10,7,2,1,23.0,4.0,19.0,23,76.7,37.7,48.0,50.0,5
7,Watford,10,4,3,3,15.0,18.0,-3.0,15,50.0,47.7,46.2,47.9,6
5,Arsenal,10,6,1,3,19.0,13.0,6.0,19,63.3,39.3,49.2,47.8,7
10,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.2,46.7,47.3,8
14,Stoke,10,3,2,5,11.0,20.0,-9.0,11,36.7,52.5,44.3,46.5,9


In [42]:
pd.set_option('precision', 1)
request_date = dt.datetime.today().strftime("%Y-%m-%d")
PLtableRPI, results_date = gen_prem_table_RPI(before_date=request_date, update_cache=False)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-10-30, date of results data: 2017-10-29


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 30 Oct 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35.0,6.0,29.0,28,93.3,38.8,49.9,55.2,1
6,Liverpool,10,4,4,2,17.0,16.0,1.0,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18.0,10.0,8.0,19,63.3,46.1,47.3,50.7,3
3,Tottenham,10,6,2,2,19.0,7.0,12.0,20,66.7,44.0,46.8,50.4,4
2,Man United,10,7,2,1,23.0,4.0,19.0,23,76.7,37.7,48.0,50.0,5
7,Watford,10,4,3,3,15.0,18.0,-3.0,15,50.0,47.7,46.2,47.9,6
5,Arsenal,10,6,1,3,19.0,13.0,6.0,19,63.3,39.3,49.2,47.8,7
10,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.2,46.7,47.3,8
14,Stoke,10,3,2,5,11.0,20.0,-9.0,11,36.7,52.5,44.3,46.5,9


In [43]:
pd.set_option('precision', 1)
request_date = '2017-10-24' # 9 games 2017
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-10-24, date of results data: 2017-10-29


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 24 Oct 17,,,,,,,,,,,,,,
1,Man City,9,8,1,0,32.0,4.0,28.0,25,92.6,38.7,51.0,55.2,1
3,Tottenham,9,6,2,1,19.0,6.0,13.0,20,74.1,40.7,48.5,51.0,2
4,Chelsea,9,5,1,3,17.0,10.0,7.0,16,59.3,48.1,47.2,50.7,3
9,Liverpool,9,3,4,2,14.0,16.0,-2.0,13,48.1,55.6,43.2,50.6,4
6,Watford,9,4,3,2,15.0,17.0,-2.0,15,55.6,48.6,45.1,49.5,5
8,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.3,47.6,47.6,6
2,Man United,9,6,2,1,22.0,4.0,18.0,20,74.1,33.3,49.5,47.6,7
5,Arsenal,9,5,1,3,17.0,12.0,5.0,16,59.3,39.9,49.4,47.1,8
18,Everton,9,2,2,5,7.0,18.0,-11.0,8,29.6,56.0,44.1,46.4,9


In [45]:
pd.set_option('precision', 1)
request_date = '2017-10-31' # 10 games 2017 (with 2 teams with 9 games as Burnley vs Newcasle listed but not played)
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-10-31, date of results data: 2017-10-29


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 31 Oct 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35.0,6.0,29.0,28,93.3,38.8,49.9,55.2,1
6,Liverpool,10,4,4,2,17.0,16.0,1.0,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18.0,10.0,8.0,19,63.3,46.1,47.3,50.7,3
3,Tottenham,10,6,2,2,19.0,7.0,12.0,20,66.7,44.0,46.8,50.4,4
2,Man United,10,7,2,1,23.0,4.0,19.0,23,76.7,37.7,48.0,50.0,5
7,Watford,10,4,3,3,15.0,18.0,-3.0,15,50.0,47.7,46.2,47.9,6
5,Arsenal,10,6,1,3,19.0,13.0,6.0,19,63.3,39.3,49.2,47.8,7
10,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.2,46.7,47.3,8
14,Stoke,10,3,2,5,11.0,20.0,-9.0,11,36.7,52.5,44.3,46.5,9


In [46]:
#Note: results after 10 games in 2016...
Image(url= "https://tomkinstimes.com/wp-content/uploads/2016/11/RPI-by-Tim-OBrien.png")

## Building The Spyre App

Useful reference material:
+ How to develop a Spyre app, including tutorials - [https://github.com/adamhajari/spyre](https://github.com/adamhajari/spyre).

Spyre is a web app framework for providing a simple user interface for Python data projects. In simple terms the premrpi app involves:
1. creating a user interface to call the getData() function to return a data table.
2. using pandas (and related modules) to generate the data table.

See [premrpi_app.py in the premrpi github repo](https://github.com/terrydolan/premrpi) for the app source code.